In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过移动到Vertex AI Gemini提示和调整来定制分类、实体提取和情感分析模型。无法继续使用Vertex AI AutoML来训练或更新文本分类、实体提取和情感分析模型。您可以继续使用现有的Vertex AI AutoML文本目标，直到2025年6月15日。有关Gemini通过改进的提示功能提供增强用户体验的更多信息，请参阅 [调整介绍](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-gemini-overview)。

# Vertex AI：Vertex AI迁移：AutoML文本情感分析

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-sentiment-analysis-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-sentiment-analysis-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-text-sentiment-analysis-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

这个笔记本演示了如何使用 Vertex AI ncbi 疾病研究数据集创建一个 AutoML 文本情感分析模型，以及如何为批量预测提供模型服务。您需要提供一个存储数据集的存储桶。

注意：在测试这个 SDK 时，您可能需要为训练、预测、存储或使用其他 Google Cloud 产品而产生费用。

了解更多关于 [迁移到 Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai) 和[文本数据情感分析](https://cloud.google.com/vertex-ai/docs/training-overview#sentiment_analysis_for_text)。

### 目标

这个笔记本的目标是建立一个自动机器学习文本情感分析模型。以下步骤已经完成：
本教程使用以下Google Cloud ML服务：

* Vertex AI数据集资源
* AutoML训练
* Vertex AI模型资源
* Vertex AI批量预测

已执行的步骤包括：

* 复制AutoML视频演示训练数据以创建托管数据集
* 在Vertex AI上创建一个数据集。
* 配置训练任务
* 启动训练作业并在Vertex AI上创建模型
* 复制AutoML视频演示预测数据以创建批量预测作业
* 在模型上执行批量预测作业

### 数据集

本教程使用的数据集是来自[data.world数据集](https://data.world)中的[Crowdflower Claritin-Twitter数据集](https://data.world/crowdflower/claritin-twitter)。本教程中使用的数据集版本存储在公共云存储桶中。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储 SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南中关于[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简化的指令集：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook面板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform google-cloud-storage tensorflow $USER_FLAG -q

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

此教程不需要 GPU 运行时。

### 设置您的 Google 云项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得用于计算/存储成本的 300 美元免费信用额度。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、计算引擎 API 和云存储。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并插值以 `$` 为前缀的 Python 变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您也可以更改“REGION”变量，该变量用于本笔记本中的操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您在进行现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免在资源上创建的用户之间的名称冲突，您为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

###验证您的Google云帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过身份验证。

**如果您正在使用Colab**，请运行下方的单元格，并按提示进行操作，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

点击**创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐户授予项目访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”，并选择**Vertex管理员**。在筛选框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到本地环境。

输入您的服务帐户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必需的。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时，运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过查看其内容来验证对您的云存储存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常数

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/claritin.csv"
SENTIMENT_MAX = 4

#### 快速查看您的数据

本教程使用存储在公共云存储桶中的一份Crowdflower Claritin-Twitter数据集的版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来统计示例数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建一个数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟。

In [ ]:
dataset = aip.TextDataset.create(
    display_name="Crowdflower Claritin-Twitter" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.sentiment,
)

print(dataset.resource_name)

示例输出：

    信息：google.cloud.aiplatform.datasets.dataset:创建TextDataset
    信息：google.cloud.aiplatform.datasets.dataset:创建TextDataset备份LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/3193181053544038400
    信息：google.cloud.aiplatform.datasets.dataset:TextDataset已创建。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    信息：google.cloud.aiplatform.datasets.dataset:在另一个会话中使用此TextDataset：
    信息：google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/759209241365/locations/us-central1/datasets/3704325042721521664')
    信息：google.cloud.aiplatform.datasets.dataset:导入TextDataset数据：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    信息：google.cloud.aiplatform.datasets.dataset:导入TextDataset数据备份LRO：projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/5152246891450204160
    信息：google.cloud.aiplatform.datasets.dataset:TextDataset数据已导入。资源名称：projects/759209241365/locations/us-central1/datasets/3704325042721521664
    projects/759209241365/locations/us-central1/datasets/3704325042721521664

训练一个模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### [训练自动机器学习 API](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行该管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，则为单标签（False）或多标签（True）。
- `sentiment_max`：如果是情感分析任务，则为最大情感值。

实例化的对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="claritin_" + UUID,
    prediction_type="sentiment",
    sentiment_max=SENTIMENT_MAX,
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLTextTrainingJob object at 0x7fc3b6c90f10> 

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLTextTrainingJob 对象在 0x7fc3b6c90f10>

运行训练流程

接下来，您可以通过调用方法`run`来运行DAG以启动训练作业，方法中包括以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。

完成`run`方法后，会返回`Model`资源。

训练流程的执行时间最多需要20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="claritin_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

示例输出：

     信息：google.cloud.aiplatform.training_jobs:查看训练：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/8859754745456230400?project=759209241365
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/8859754745456230400当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/8859754745456230400当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/8859754745456230400当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/8859754745456230400当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob项目/759209241365/locations/us-central1/trainingPipelines/8859754745456230400当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    …
     信息：google.cloud.aiplatform.training_jobs：AutoMLTextTrainingJob运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400
     信息：google.cloud.aiplatform.training_jobs：模型可在项目/759209241365/locations/us-central1/models/6389525951797002240找到

评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 查看模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=claritin_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

示例输出：

    名称："projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    指标架构URI："gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    指标 {
      结构体值 {
        字段 {
          键："auPrc"
          值 {
            数值: 0.9891107
          }
        }
        字段 {
          键："confidenceMetrics"
          值 {
            列表值 {
              值 {
                结构体值 {
                  字段 {
                    键："precision"
                    值 {
                      数值: 0.2
                    }
                  }
                  字段 {
                    键："recall"
                    值 {
                      数值: 1.0
                    }
                  }
                }
              }

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [批量预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### 获取测试项

现在对您的 Vertex 模型进行批量预测。您可以从数据集中随机选择示例作为测试项。不必担心这些示例可能已经用于训练模型 — 我们只是想展示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 4:
    _, test_item_1, test_label_1, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

制作批量输入文件

现在制作一个批处理输入文件，将其存储在您的本地云存储桶中。 批处理输入文件只能以JSONL格式存在。 对于JSONL文件，您为每个数据项（实例）在一行中制作一个字典条目。 字典包含键/值对：

- `content`：包含文本项的文件的云存储路径。
- `mime_type`：内容类型。 在我们的例子中，是一个`text`文件。

例如：

                         {'content': '[您的桶]/file1.txt'， 'mime_type'：'text' }

In [ ]:
import json

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

进行批量预测请求

现在您的模型资源已经训练完毕，您可以通过调用batch_predict()方法进行批量预测，需要以下参数：
- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，调用会在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="claritin_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

*示例输出：*

INFO：google.cloud.aiplatform.jobs：创建BatchPredictionJob
<google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
INFO：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
INFO：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
INFO：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
INFO：google.cloud.aiplatform.jobs：查看Batch Prediction Job：
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
JobState.JOB_STATE_RUNNING

### 等待批处理预测任务完成

接下来，等待批处理任务完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测任务完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    INFO: google.cloud.aiplatform.jobs：已创建BatchPredictionJob。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO: google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    INFO: google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO: google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    INFO: google.cloud.aiplatform.jobs：BatchPredictionJob运行已完成。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测

接下来，从已完成的批量预测作业中获取结果。

结果将被写入你在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs（）方法获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `sentiment`：情感。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

示例输出：

{'实例': {'内容': 'gs://andy-1234-221921aip-20210811220920/test2.txt', 'mimeType': 'text/plain'}, '预测': {'情绪': 3}}

做线上预测

### [预测部署模型 API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

示例输出：

    信息：google.cloud.aiplatform.models:创建端点
    信息：google.cloud.aiplatform.models:创建端点支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    信息：google.cloud.aiplatform.models:端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:在另一个会话中使用此端点：
    信息：google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    信息：google.cloud.aiplatform.models:将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:部署端点模型支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    信息：google.cloud.aiplatform.models:端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

[预测。在线预测AutoML](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

获取测试项目

您可以从数据集中随意选择一个示例作为测试项目。不用担心这个示例很可能在训练模型时被使用过--我们只是想演示如何进行预测。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label, max = str(test_item[0]).split(",")
else:
    test_item, test_label, max = str(test_item[0]).split(",")

print(test_item, test_label)

进行预测

现在，您的`Model`资源已部署到`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

     { 'content': text_string }

由于predict()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

predict()调用的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `sentiment`：情感值。
- `deployed_model_id`：进行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

*示例输出：*

    预测(predictions=[{'sentiment': 2.0}], 部署的模型ID='311601595311718400', 说明=None)

取消部署模型

当您完成预测后，您可以从`Endpoint`资源中取消部署模型。这将取消所有计算资源的供应并结束已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud 存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_URI